# All D11 Calculation Pathways

This notebook analyzes snow D11 (flexural rigidity in the x-direction) calculation methods at the **slab-level** scale.

## Table of Contents

1. [Load Snow Pit Data](#1-load-snow-pit-data)
2. [Find All D11 Calculation Pathways](#2-find-all-d11-calculation-pathways)
3. [Slab-Level Analysis (ECTP)](#3-slab-level-analysis-ectp)

**Target Parameter**: `D11` — slab flexural rigidity in the x-direction (N·m)

D11 is a slab-level property requiring elastic modulus and Poisson's ratio for all layers. Uncertainty reflects propagated input measurement uncertainties only (method regression standard error excluded).

In [1]:
from pathlib import Path
from typing import Dict, Any
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

from snowpyt_mechparams.snowpilot import parse_caaml_directory
from snowpyt_mechparams.data_structures import Pit, Slab
from snowpyt_mechparams.graph import graph
from snowpyt_mechparams.algorithm import find_parameterizations
from snowpyt_mechparams.execution import ExecutionEngine
from snowpyt_mechparams.execution.config import ExecutionConfig

## 1. Load Snow Pit Data

In [2]:
snow_pits_raw = parse_caaml_directory(str(Path("data")))
pits = [Pit.from_snow_pit(sp) for sp in snow_pits_raw]

print(f"Loaded {len(pits)} snow pits ({sum(len(pit.layers) for pit in pits)} layers)")

Loaded 50278 snow pits (371429 layers)


## 2. Find All D11 Calculation Pathways

In [3]:
pathways = find_parameterizations(graph, graph.get_node("D11"))

print(f"Found {len(pathways)} pathways for calculating D11:\n")
for i, pathway in enumerate(pathways, 1):
    print(f"Pathway {i}:")
    print(pathway)
    print()

Found 32 pathways for calculating D11:

Pathway 1:
branch 1: snow_pit -- data_flow --> measured_layer_thickness -- data_flow --> zi
branch 2: snow_pit -- data_flow --> measured_density -- data_flow --> density -- data_flow --> merge_density_grain_form
branch 3: snow_pit -- data_flow --> measured_grain_form -- data_flow --> merge_density_grain_form
branch 4: snow_pit -- data_flow --> measured_grain_form -- kochle --> poissons_ratio -- data_flow --> merge_E_nu
merge branch 1: zi
merge branch 2, branch 3: merge_density_grain_form -- bergfeld --> elastic_modulus
merge branch 2, branch 3, branch 4: merge_E_nu
merge branch 1, branch 2, branch 3, branch 4: merge_zi_E_nu -- weissgraeber_rosendahl --> D11

Pathway 2:
branch 1: snow_pit -- data_flow --> measured_layer_thickness -- data_flow --> zi
branch 2: snow_pit -- data_flow --> measured_density -- data_flow --> density -- data_flow --> merge_density_grain_form
branch 3: snow_pit -- data_flow --> measured_grain_form -- data_flow --> merge_de

### Why 32 Pathways, Not More?

D11 requires three layer-level parameters, each with multiple methods:

| Parameter | Methods | Count |
|-----------|---------|-------|
| `density` | `data_flow`, `geldsetzer`, `kim_jamieson_table2`, `kim_jamieson_table5` | 4 |
| `elastic_modulus` | `bergfeld`, `kochle`, `wautier`, `schottner` | 4 |
| `poissons_ratio` | `kochle`, `srivastava` | 2 |

4 × 4 × 2 = **32 unique method combinations**.

The `srivastava` Poisson's ratio method requires `density` as input. The algorithm's backward traversal treats the density sub-path for `elastic_modulus` and for `srivastava` as independent choices, generating up to 80 structural traversals. `find_parameterizations` deduplicates them using `_method_fingerprint`, leaving exactly the 32 unique combinations above.

## 3. Slab-Level Analysis (ECTP)

In [4]:
# Create ECTP slabs
ectp_slabs = []
for pit in pits:
    for slab in pit.create_slabs(weak_layer_def="ECTP_failure_layer"):
        ectp_slabs.append({'slab': slab, 'n_layers': len(slab.layers)})

print(f"Created {len(ectp_slabs)} ECTP slabs")

Created 14776 ECTP slabs


In [5]:
engine = ExecutionEngine(graph)
config = ExecutionConfig(include_method_uncertainty=False)

# Execute all D11 pathways on each ECTP slab
d11_data = []
pathway_slab_success: Dict[str, int] = {}
pathway_rel_unc: Dict[str, list] = {}

for info in ectp_slabs:
    slab = info['slab']
    results = engine.execute_all(slab, "D11", config=config)
    for pathway_result in results.pathways.values():
        density_method = pathway_result.methods_used.get('density', 'unknown')
        e_mod_method = pathway_result.methods_used.get('elastic_modulus', 'unknown')
        pr_method = pathway_result.methods_used.get('poissons_ratio', 'unknown')
        full_pathway = f"{density_method} \u2192 {e_mod_method} \u2192 {pr_method}"

        d11_val = None
        d11_std = None
        for trace in pathway_result.computation_trace:
            if trace.parameter == "D11" and trace.success and trace.output is not None:
                out = trace.output
                if hasattr(out, 'nominal_value'):
                    d11_val, d11_std = out.nominal_value, out.std_dev
                else:
                    try:
                        d11_val, d11_std = float(out), 0.0
                    except (TypeError, ValueError):
                        pass
                break

        if full_pathway not in pathway_slab_success:
            pathway_slab_success[full_pathway] = 0
            pathway_rel_unc[full_pathway] = []

        if d11_val is not None:
            pathway_slab_success[full_pathway] += 1
            if d11_val != 0 and d11_std is not None:
                pathway_rel_unc[full_pathway].append(d11_std / d11_val)

print(f"Executed {len(pathways)} pathways on {len(ectp_slabs)} slabs")

Executed 32 pathways on 14776 slabs


In [ ]:
total_slabs = len(ectp_slabs)

# Sort by slab count descending
sorted_pathways = sorted(pathway_slab_success.items(), key=lambda x: x[1], reverse=True)

print(f"  {'Full Pathway':<55s} {'Slabs':>30s} {'Avg Rel. Uncertainty':>22s}")
print(f"  {'-'*109}")
for pathway, n_ok in sorted_pathways:
    slab_str = f"{n_ok} / {total_slabs} ({n_ok / total_slabs:.1%})"
    rel_uncs = pathway_rel_unc.get(pathway, [])
    avg_unc = np.mean(rel_uncs) if rel_uncs else float('nan')
    unc_str = f"{avg_unc:.1%}" if not np.isnan(avg_unc) else "N/A"
    print(f"  {pathway:<55s} {slab_str:>30s}    {unc_str:>18s}")

print()
print("  Note: Uncertainty is propagated from input measurement uncertainties only.")
print("  Slab success requires D11 calculation for the entire slab.")


  Full Pathway                                                 Slabs       (Success Rate)   Avg Rel. Uncertainty
  ------------------------------------------------------------------------------------------------------
  geldsetzer → wautier → kochle                              745 / 14776  ( 5.0%)                 41.7%
  kim_jamieson_table2 → wautier → kochle                     745 / 14776  ( 5.0%)                 40.8%
  geldsetzer → schottner → kochle                            745 / 14776  ( 5.0%)                 74.8%
  kim_jamieson_table2 → schottner → kochle                   745 / 14776  ( 5.0%)                 73.2%
  geldsetzer → kochle → kochle                               500 / 14776  ( 3.4%)                 87.8%
  kim_jamieson_table2 → bergfeld → kochle                    498 / 14776  ( 3.4%)                 75.4%
  geldsetzer → bergfeld → kochle                             476 / 14776  ( 3.2%)                 76.7%
  geldsetzer → wautier → srivastava                   